# Notebook ICD - 7

Install libraries via terminal

In [1]:
#pip install spacy
#pip install nltk
#pip install sklearn

Se importan las librer&iacute;as necesarias

In [2]:
import pandas as pd
import re
import spacy as spc
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

## Datos de entrada

In [27]:
oracion = "Las futbolistas se entrenan en Gotemburgo tras la comparecencia de las capitanas"
print(oracion)
oracion1 = " Un interruptor multitarea permite cambiar la atención de una tarea a otra"
print(oracion1)

Las futbolistas se entrenan en Gotemburgo tras la comparecencia de las capitanas
 Un interruptor multitarea permite cambiar la atención de una tarea a otra


## Procesamiento

### Limpieza de datos

In [28]:
oracion = oracion.lower()
oracion = re.sub(r"@\S+", "", oracion)  # Eliminar menciones a usuarios
oracion = re.sub("http[s]?\://\S+", "", oracion)  # Eliminar enlaces
oracion = re.sub(r"#\S+", "", oracion)  # Eliminar hashtags
oracion = re.sub(r"[0-9]", "", oracion)  # Eliminar números
oracion = re.sub(r"(\(.*\))|(\[.*\])", "", oracion)  # Eliminar paréntesis y corchetes
oracion = re.sub(r"\n", "", oracion)  # Eliminar caracteres de nueva línea
oracion = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", "", oracion)  # Eliminar varios patrones
oracion = re.sub(r"(\.)|(,)", "", oracion)  # Eliminar puntos y comas
oracion = re.sub(r"[¡!]", "", oracion)  # Eliminar signos de admiración 
oracion = re.sub(r"[¿?]", "", oracion)  # Eliminar signos de exclamación
print(oracion)

oracion1 = oracion1.lower()
oracion1 = re.sub(r"\n", "", oracion1)  # Eliminar caracteres de nueva línea
print(oracion1)


las futbolistas se entrenan en gotemburgo tras la comparecencia de las capitanas
 un interruptor multitarea permite cambiar la atención de una tarea a otra


### Tokenización

In [29]:
#nltk.download('punkt')
tokens = word_tokenize(oracion)
print(tokens)

tokens1 = word_tokenize(oracion1)
print(tokens1)

['las', 'futbolistas', 'se', 'entrenan', 'en', 'gotemburgo', 'tras', 'la', 'comparecencia', 'de', 'las', 'capitanas']
['un', 'interruptor', 'multitarea', 'permite', 'cambiar', 'la', 'atención', 'de', 'una', 'tarea', 'a', 'otra']


### Stopwords

In [30]:
#nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')
palabras_filtradas = [palabra for palabra in tokens if palabra not in spanish_stopwords]
print(palabras_filtradas)

spanish_stopwords1 = stopwords.words('spanish')
palabras_filtradas1 = [palabra for palabra in tokens1 if palabra not in spanish_stopwords]
print(palabras_filtradas1)

['futbolistas', 'entrenan', 'gotemburgo', 'tras', 'comparecencia', 'capitanas']
['interruptor', 'multitarea', 'permite', 'cambiar', 'atención', 'tarea']


### Lematización

In [7]:
#sudo python3 -m spacy download es

In [31]:
nlp = spc.load("es_core_news_sm")
lema = nlp(" ".join(palabras_filtradas))
oracion_lematizada = " ".join([token.lemma_ for token in lema])
print(oracion_lematizada)

nlp = spc.load("es_core_news_sm")
lema = nlp(" ".join(palabras_filtradas1))
oracion_lematizada1 = " ".join([token.lemma_ for token in lema])
print(oracion_lematizada1)

futbolista entrenar gotemburgo tras comparecencia capitana
interruptor multitarea permitir cambiar atención tareo


## Bag of Words

In [32]:
vectorizador = CountVectorizer()
vectores = vectorizador.fit_transform([oracion_lematizada])
vocabulario = vectorizador.get_feature_names_out()

vectorizador1 = CountVectorizer()
vectores1 = vectorizador1.fit_transform([oracion_lematizada1])
vocabulario1 = vectorizador1.get_feature_names_out()

### Resultados

In [33]:
print("Oración de entrada:", oracion)
print("Oración lematizada:", oracion_lematizada)
print("Vectores Bag of Words:", vectores.toarray())
print("Vocabulario:", vocabulario)

print("Oración de entrada:", oracion1)
print("Oración lematizada:", oracion_lematizada1)
print("Vectores Bag of Words:", vectores1.toarray())
print("Vocabulario:", vocabulario1)

Oración de entrada: las futbolistas se entrenan en gotemburgo tras la comparecencia de las capitanas
Oración lematizada: futbolista entrenar gotemburgo tras comparecencia capitana
Vectores Bag of Words: [[1 1 1 1 1 1]]
Vocabulario: ['capitana' 'comparecencia' 'entrenar' 'futbolista' 'gotemburgo' 'tras']
Oración de entrada:  un interruptor multitarea permite cambiar la atención de una tarea a otra
Oración lematizada: interruptor multitarea permitir cambiar atención tareo
Vectores Bag of Words: [[1 1 1 1 1 1]]
Vocabulario: ['atención' 'cambiar' 'interruptor' 'multitarea' 'permitir' 'tareo']


### Construir Dataframe

In [34]:
df_bw = pd.DataFrame.sparse.from_spmatrix(vectores,columns = vocabulario)
df_bw.head()



,capitana,comparecencia,entrenar,futbolista,gotemburgo,tras
0,1,1,1,1,1,1


In [35]:
df_bw['Clase'] = 'Deporte'

In [36]:
df_bw1 = pd.DataFrame.sparse.from_spmatrix(vectores1,columns = vocabulario1)
df_bw1.head()

,atención,cambiar,interruptor,multitarea,permitir,tareo
0,1,1,1,1,1,1


In [37]:
df_bw1['Clase'] = 'Frase'

In [38]:
resultado = pd.merge(df_bw,df_bw1, how="outer", on="Clase")
resultado = resultado [[col for col in resultado.columns if col != 'Clase'] + ['Clase']]
resultado.fillna(0)


,capitana,comparecencia,entrenar,futbolista,gotemburgo,tras,atención,cambiar,interruptor,multitarea,permitir,tareo,Clase
0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Deporte
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,Frase
